# AI and Machine Learning Coursework

Use the provided retinal image datasets (classified as normal, cataract or glaucoma) to develop a machine learning model that predicts the disease status of an image. Apply your knowledge from previous workshops to design, implement, and evaluate the model.

This was done in Python 3.11.9

## BASIC PREPROCESSING

In [ ]:
# import libraries
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow
import sys
# File path for retina images
path = 'path/Images'

In [ ]:
# make file paths for each image class
Normal = os.path.join(path,'1_normal')
Cataract = os.path.join(path,'2_cataract')
Glaucoma = os.path.join(path,'2_glaucoma')
# print out how many images are in each class
print("Number of normal retinas", len(os.listdir(Normal)))
print("Number of cataract retinas", len(os.listdir(Cataract)))
print("Number of glaucoma retinas", len(os.listdir(Glaucoma)))

In [1]:
#Function for loading images
def load_images(directories, n_images=900000):
    """
    Reads in images and assigns class labels
    Parameters:
        directories: A list of the sub-directories
        n_images:    The maximum number of images to load from each directory
    Returns:
        images (numpy.ndarray) : Image data
        label (numpy.ndarray      : Labels of each image
    """
    images = []
    labels = []
    for label, sub_dir in enumerate(directories):
        num=1
        for file_name in os.listdir(sub_dir):
            if num > n_images:
                break
            img_path = os.path.join(sub_dir, file_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (500, 500))  # Resize to a smaller, consistent shape
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)
                labels.append(label)
                num+=1
    return np.array(images), np.array(labels)

In [ ]:
#load images into python
images, labels = load_images([Normal, Cataract, Glaucoma], 600)

In [ ]:
#check to make sure images have properly loaded in
print(images.shape)
# display an image to see if it has loaded in correctly
plt.imshow(images[296],cmap="gray")
#python counts from 0 so image 297 in folder is actually 296

In [ ]:
# split data into training and test data, 20% is testing data with a random seed set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

## MODEL DEVELOPMENT

In [ ]:
#import DenseNet model
from tensorflow.keras.applications import DenseNet169
#using Resnet101V2 model
densenet_model = DenseNet169(
    weights='imagenet',
    include_top=False,
    input_shape= (500,500,3),
)

# freeze the ResNet convolutional layers to add extra trainable layers to the ResNet NN so we can train on our data

for layer in densenet_model.layers:
    layer.trainable = False

In [ ]:
#setup early stopping after 5 epochs of no improvement to loss
from tensorflow.keras.callbacks import EarlyStopping
callback = tensorflow.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='auto',
    restore_best_weights=False,
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Rescaling, Input, RandAugment
from tensorflow.keras.initializers import HeNormal
initializer = HeNormal()

model = Sequential( [
 Input(shape=(500, 500, 3)),
 tensorflow.keras.layers.Rescaling(1./255),
 tensorflow.keras.layers.RandomContrast(factor=0.1, value_range= [0,255]),
 #tensorflow.keras.layers.RandomBrightness(factor=0.2, value_range= [0,255]), #this augmentation massively decreases performance for some reason, probably configured incorrectly
 tensorflow.keras.layers.RandomFlip("horizontal"),
 tensorflow.keras.layers.RandomRotation((-0.1,0.1)), # % of 2*PI radians rotation 
 tensorflow.keras.layers.RandomZoom(0.1),
 tensorflow.keras.layers.RandomTranslation(height_factor=(-0.1,0.1), width_factor=(-0.1,0.1)),
 densenet_model, 
 Flatten(),
 Dense(128, activation='relu'),  # Put into a dense layer size
 Dense(64, activation='relu'),  # Put into a dense layer size
 Dense(3, activation='softmax')  # 3 state classification - gives probability of belonging to each class
] )

In [ ]:
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.losses import CategoricalCrossentropy

#compile model
model.compile(optimizer=Adamax(learning_rate= 0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy', 'categorical_accuracy'])
# Categorical Crossentropy is designed for multi-class classifications with multiple class outputs.
# print model summary
model.summary()

### RESAMPLING

In [ ]:
#SMOTE-Tomek Links Over and undersampling
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
#need to reshape data to work with resampling functions
reshaped_X = X_train.reshape(X_train.shape[0],-1)

#resampling with SMOTE and Tomek links
smote_object = SMOTE(sampling_strategy='not majority', random_state=69, k_neighbors=5)
tomek_object = TomekLinks(sampling_strategy = 'majority')
resample = SMOTETomek(sampling_strategy = 'not majority', smote = smote_object, tomek = tomek_object, random_state = 69 )
resampled_X, resampled_y  = resample.fit_resample(reshaped_X , y_train)

# reshaping back to the initial dimensions
new_X_train = resampled_X.reshape(-1,500,500,3)
new_y_train = resampled_y.reshape(-1,)

In [ ]:
import tensorflow
# one hot encoding for categorical crossentropy
y_train_cat = tensorflow.keras.utils.to_categorical(new_y_train, num_classes=3)
# also for the test set
y_test_cat = tensorflow.keras.utils.to_categorical(y_test, num_classes=3)

In [ ]:
#check that the one hot encoding and resampling worked 
y_train_cat.shape

## MODEL TRAINING

In [ ]:
#model training
history = model.fit(x=new_X_train, y=y_train_cat, batch_size=50,
                      epochs=30, shuffle=True, 
                      validation_split=0.2, callbacks = callback)

In [ ]:
#optionally save the model for later
model.save('path_to_directory/Retina_model2.keras')

## CROSS VALIDATE

In [ ]:
#optionally load in the saved pretrained model and its weights if you somehow lost the saved model before
model = tensorflow.keras.models.load_model('path_to_saved_model/Retina_model2.keras')

model.summary()

In [ ]:
#cross validation with Stratified K-fold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from scikeras.wrappers import KerasClassifier
#setup cross validation object
validate = StratifiedKFold(n_splits=5, shuffle= True)
#tensorflow/keras model needs to be wrapped for sklearn to process it
keras_model = KerasClassifier(model= model, loss= 'categorical_crossentropy', optimizer='Adamax', epochs = 30, batch_size= 50)
#deisgnate metrics to evaluate
metrics = {'acc': 'accuracy',
           'prec_macro': 'precision_macro', 'prec_weight': 'precision_weighted',
           'f1_macro':'f1_macro', 'f1_weight': 'f1_weighted', 
           'recall_macro':'recall_macro', 'recall_weight': 'recall_weighted',
           'roc': 'roc_auc_ovr', 'roc_weighted': 'roc_auc_ovr_weighted'}

#run cross validation (can take several hours, very memory intensive)
scores = cross_validate(keras_model, images, labels, scoring= metrics, cv=validate, n_jobs=5, error_score="raise")
print(scores)

## MODEL EVALUATION

### Accuracy and Loss graphs

In [ ]:
# accuracy changes during training as a graph
epochs = history.epoch
accuracy_values = history.history['accuracy']
val_accuracy_values = history.history['val_accuracy']

plt.figure(figsize=(8, 4))
plt.plot(epochs, accuracy_values, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy_values, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# loss changes during training as a graph
loss_values = history.history['loss']
val_loss_values = history.history['val_loss']

plt.figure(figsize=(8, 4))
plt.plot(epochs, loss_values, 'b', label='Training Loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Testing on test set

In [ ]:
# Use the model to make predictions on the testset
y_pred_prob = model.predict(X_test)
# Convert predictions to binary class labels 
y_pred  = y_pred_prob.argmax(axis=1)

In [ ]:
#make confusion matrix from predictions
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm  = confusion_matrix(y_test, y_pred)
cmdisp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(5, 5))
cmdisp.plot(include_values=True, cmap="viridis", ax=ax, xticks_rotation="vertical")
plt.show()

In [ ]:
#print classification table
from sklearn.metrics  import classification_report
print('Classification Report:')
print(classification_report(y_test,y_pred))

### Precision Recall Curve

In [ ]:
#data needs to be transformed to work with the plots
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
#calculate PR curves for each class
from sklearn.metrics import average_precision_score, precision_recall_curve
n_classes = 3

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_onehot_test[:, i], y_pred_prob[:, i])
    average_precision[i] = average_precision_score(y_onehot_test[:, i], y_pred_prob[:, i])


In [ ]:
from itertools import cycle
from sklearn.metrics import PrecisionRecallDisplay
import matplotlib.pyplot as plt

# setup plot details
colors = cycle(["navy", "turquoise", "darkorange", "cornflowerblue", "teal"])

_, ax = plt.subplots(figsize=(7, 8))

f_scores = np.linspace(0.2, 0.8, num=4)
lines, labels = [], []

for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(
        ax=ax, name=f"Precision-recall for class {i}", color=color
    )

# add the legend for the iso-f1 curves
handles, labels = display.ax_.get_legend_handles_labels()
handles.extend([l])
# set the legend and the axes
ax.legend(handles=handles, labels=labels, loc="best")
ax.set_title("Precision-Recall curve of every Retina Image class")

plt.show()

### ROC AUC PLOTS

In [ ]:
#setup objects beforehand
fpr, tpr, roc_auc = dict(), dict(), dict()

In [ ]:
#calculate macro averaged ROC AUC score
macro_roc_auc_ovr = roc_auc_score(
    y_test,
    y_pred_prob,
    multi_class="ovr",
    average="macro",
)

print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{macro_roc_auc_ovr:.2f}")

In [ ]:
#calculate micro averaged ROC AUC score
fpr["micro"], tpr["micro"], _ = roc_curve(y_onehot_test.ravel(), y_pred_prob.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{roc_auc['micro']:.2f}")

In [ ]:
#plot ROC AUC graph 
from itertools import cycle
n_classes = 3
target_names = ('Normal', 'Cataract', 'Glaucoma')
fig, ax = plt.subplots(figsize=(6, 6))

#micro ROC AUC curve
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label=f"micro-average ROC curve (AUC = {roc_auc['micro']:.2f})",
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

#macro ROC AUC curve
plt.plot(
    fpr["macro"],
    tpr["macro"],
    label=f"macro-average ROC curve (AUC = {roc_auc['macro']:.2f})",
    color="navy",
    linestyle=":",
    linewidth=4,
)

#ROC AUC curve for each class
colors = cycle(["aqua", "darkorange", "cornflowerblue"])
for class_id, color in zip(range(n_classes), colors):
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_pred_prob[:, class_id],
        name=f"ROC curve for {target_names[class_id]}",
        color=color,
        ax=ax,
        plot_chance_level=(class_id == 2),
    )
#setting axis labels and titles
_ = ax.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="Receiver Operating Characteristic\nto One-vs-Rest multiclass of Retina Images",
)